
# Seminararbeit

##  Das verschlüsselte im Offensichtlichen: Emotionen


_______________________________________________________________________________________________________________________________________________________

### Emotion Facial Recognition of Drawn Characters: Transfer Learning Task

_______________________________________________________________________________________________________________________________________________________

In [1]:
from google.colab import drive
drive.mount("/content/drive")

path_folder = "/content/drive/MyDrive/Seminararbeit/Trainingsbilder"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Imports

In [2]:
import numpy as np
from matplotlib import pyplot as plt
import torch
import torchvision
from torchvision import transforms as T

from scipy import ndimage

import cv2 #for image processing
# import easygui #to open the filebox
import numpy as np #to store image
# import imageio #to read image stored at particular path
import sys
import matplotlib.pyplot as plt
import os
import tkinter as tk
from tkinter import filedialog
from tkinter import *
from PIL import ImageTk, Image

from tqdm import tqdm
import collections

In [3]:
targets = ["anger","contempt","disgust","fear","happiness","neutrality","sadness","surprise"]

# A vector representation of the target for training later
tar_name2vec= {
    "anger"     : torch.tensor([1,0,0,0,0,0,0,0], dtype=torch.float32) ,
    "contempt"  : torch.tensor([0,1,0,0,0,0,0,0], dtype=torch.float32) ,
    "disgust"   : torch.tensor([0,0,1,0,0,0,0,0], dtype=torch.float32) ,
    "fear"      : torch.tensor([0,0,0,1,0,0,0,0], dtype=torch.float32) ,
    "happiness" : torch.tensor([0,0,0,0,1,0,0,0], dtype=torch.float32) ,
    "neutrality": torch.tensor([0,0,0,0,0,1,0,0], dtype=torch.float32) ,
    "sadness"   : torch.tensor([0,0,0,0,0,0,1,0], dtype=torch.float32) ,
    "surprise"  : torch.tensor([0,0,0,0,0,0,0,1], dtype=torch.float32) 
}

def tar_vec2name(vec):
    if vec[0] == 1: 
        return "anger" 

    elif vec[1] == 1:
        return "contempt"
    
    elif vec[2] == 1:
        return "disgust"

    elif vec[3] == 1:
        return "fear"

    elif vec[4] == 1:
        return "happines"

    elif vec[5] == 1:
        return "neutrality"

    elif vec[6] == 1:
        return "sadness"

    elif vec[7] == 1:
        return "surprise"

for i in targets:
    print(tar_vec2name(tar_name2vec[i]))


anger
contempt
disgust
fear
happines
neutrality
sadness
surprise


________________________________

## Hilfsfunktionen

In [4]:
# füge RGB hinzu
# von (y, x) zu (y, x, c)
def add_RGBLayers(im):
    if len(im.shape) == 3:
        return torch.tensor(im)
    elif len(im.shape) == 2:
        return torch.tensor(np.stack((im, im, im), 2))
    
    

In [5]:
def calc_loss(network, loader):
    network.eval()
    loss = []
    g = 0
    crit = torch.nn.BCEWithLogitsLoss(reduction="sum")
    for b_idx, (dat, tar) in enumerate(loader):
        loss.append(crit(network(dat), tar).item())
        g += len(tar)
    
    return np.sum(loss)/g

In [6]:
def max_Vektor(vec):
    x = torch.max(vec)

    idx = (vec == x).nonzero().flatten()

    t = torch.zeros_like(vec)
    t[idx] = 1
    return t

In [7]:
tar_vec2name(max_Vektor(torch.tensor([0.1473, 0.0928, 0.1254, 0.1106, 0.2664, 0.1633, 0.0834, 0.0107])))

'happines'

In [8]:
def get_different_classes(dict):
    d = {}
    
    for (dat, tar) in dict:
        if tar not in d:
            d[tar] = 1
        else:
            d[tar] += 1

    return d

In [9]:
def my_train_validation_split(dataset, test_train_split=0.8, balanced_split=True, shuffle=True, trans=None):
    
    #TODO balanced split

    dataset_size = len(dataset)
    indices = list(range(dataset_size))

    dict = {}
    for i in range(dataset_size):
        dict[i] = dataset.__getitem__(i)
        

    # if balanced_split:
    #     dclasses = get_different_classes(dict)

    trainset, testset = take_sample(dict, indices, test_train_split, shuffle)
            
    return myFaceImageDataset(trainset, transform=trans), myFaceImageDataset(testset)

    #TODO balanced split
        

In [10]:
def take_sample(dict, idx, split,shuffle=False):
    l = len(dict)
    if shuffle:
        np.random.shuffle(idx)

    train = idx[:int(split * l)]
    test = idx[int(split * l):]

    dtrain = {}
    dtest = {}
    counter = 0
    for i in train:
        dtrain[counter] = dict[i]
        counter+=1
    counter = 0
    for i in test:
        dtest[counter] = dict[i]
        counter+=1
    counter = 0

    return dtrain, dtest
    

In [11]:
#return a certain number of file paths as a set; shuffled or not shuffled
def get_filenames(path_folder, number = 1000, shuffle = True):
    names = {}
    idx = 0
    for filename in os.listdir(path_folder):

            f = os.path.join(path_folder,filename)
            if os.path.isfile(f):
                names[idx] = f
                idx += 1
    
    l = len(names)
    idx_vec = list(range(l))
    if shuffle:
        np.random.shuffle(idx_vec)
    
    if number > l: #Damit sichergestellt ist das nicht out of range gesliced werden kann
        number = l
    idx4Files_notUsed = idx_vec[number:]

    for i in idx4Files_notUsed:
        names.pop(i)

    return set(names.values())
                

In [12]:
def load_balanced_data(path_folder, max_number_per_class = 1000, shuffle = True, add_RGB = True):
    data = {} 
    count = 0

    for t in (targets):    
        p = path_folder + "\\" + t
        # print(t)
        files2get = get_filenames(p, max_number_per_class, shuffle)
        for n in tqdm(files2get):
            # the sample is save on the 0. position and the target on the 1. position
            x = torch.tensor(plt.imread(n), dtype=torch.float32)

            if add_RGB: 
                x = add_RGBLayers(x)

            data[count] = (x, tar_name2vec[t])

            count += 1

    return myFaceImageDataset(data, True)

# Image Augmentation

Das Baselinemodell overfitter stark und die unterschiedlichen Klassen sind unbalanciert. Dies beischlusst die Vorhersage sehr stark, da z.B. für die Klasse "happiness" viel mehr Updates durchgeführt werden. Außerdem würde ein größerer Datensatz einen regularisierenden Effekt haben.\

[Pytorch: Augmentation Overview](https://datamahadev.com/performing-image-augmentation-using-pytorch/)

In [13]:
# wird nicht gebraucht, da die Augmentations auch im Dataset selbst erstellt werden könne, ohne alle vorher schon abgespeichert zu haben
def augmentation_pipeline_forOneImage(original_ima):
    original_ima = original_ima.transpose(-1,0)
    

    augemntation_combis = [original_ima]

    #all augmentations
    center_crop = torchvision.transforms.RandomCrop(size=(200,200), fill=0, padding_mode="constant")
    horizontal_flip = torchvision.transforms.RandomVerticalFlip(p=1)
    random_rotation = torchvision.transforms.RandomRotation(degrees = 45)
    random_erase = torchvision.transforms.RandomErasing(p=1)

    added_augments = []
    for i in augemntation_combis:
        added_augments.append(center_crop(i)) #cv2.resize(img, (new_w, new_h))
    augemntation_combis.extend(added_augments)
    
    added_augments = []
    for i in augemntation_combis:
        added_augments.append(horizontal_flip(i))
    augemntation_combis.extend(added_augments)

    added_augments = []
    for i in augemntation_combis:
        added_augments.append(random_rotation(i))
    augemntation_combis.extend(added_augments)

    added_augments = []
    for i in augemntation_combis:
        added_augments.append(random_erase(i))
    augemntation_combis.extend(added_augments)

    #lighting the Image up a bit
    added_augments = []
    for i in augemntation_combis:
        added_augments.append(np.clip(i*1.2, 0, 1))
    augemntation_combis.extend(added_augments)

    #making the Image a bit darker
    added_augments = []
    for i in augemntation_combis:
        added_augments.append(np.clip(i*0.8, 0, 1))
    augemntation_combis.extend(added_augments)
    
    for i in range(len(augemntation_combis)):
        augemntation_combis[i] = augemntation_combis[i].transpose(-1,0)

    # damit die Dimensionalitäten bei allen Bilder gleich sind
    # Gab Problem beim Trainig, da manche Bilder 200x200x3 groß waren und andere 224x224x3
    for i in range(len(augemntation_combis)):  
        augemntation_combis[i] = augemntation_combis[i].numpy() * 255 # or any coefficient
        augemntation_combis[i] = augemntation_combis[i].astype(np.uint8)
        augemntation_combis[i] = cv2.resize(augemntation_combis[i], (224,224))


    return augemntation_combis


In [14]:
def augmentation_pipeline_forFolder(origin_path_folder, destination_path_folder, add_RGB):
    #Loading the data into a the Notebook

    for t in targets:    
        p = origin_path_folder + "\\" + t
        #print(t)
        for filename in os.listdir(p):

            f = os.path.join(p,filename)
            if os.path.isfile(f):

                # the sample is save on the 0. position and the target on the 1. position
                x = torch.tensor(plt.imread(f), dtype=torch.float32)

                #print(count)
                #print(filename)
                if add_RGB: 
                    x = add_RGBLayers(x)

                aug = augmentation_pipeline_forOneImage(x)
               
                path = f.replace(origin_path_folder, destination_path_folder)

                count = 0
                for i in aug:
                    
                    n = path.split("\\")
                    n = n[-1].rsplit(".",1)
                    newName= n[0] +str(count)
                    extension=os.path.splitext(f)[1]
                    path1 = os.path.dirname(path)
                    path2 = os.path.join(path1, newName + extension)
            
                    cv2.imwrite(path2, i)

                    count += 1
                      

### Cartoonify Funktion mit OpenCV

Source: https://data-flair.training/blogs/cartoonify-image-opencv-python/

In [15]:
def cartoonify(ImagePath):
    #read the image
    originalmage = cv2.imread(ImagePath)
    originalmage = cv2.cvtColor(originalmage, cv2.COLOR_BGR2RGB)
    #print(image)  # image is stored in form of numbers
    # confirm that image is chosen
    if originalmage is None:
        print("Can not find any image. Choose appropriate file")
        sys.exit()

    y, x, c = originalmage.shape

    ReSized1 = cv2.resize(originalmage, (x, y))
    #plt.imshow(ReSized1, cmap='gray')

    #converting an image to grayscale
    grayScaleImage = cv2.cvtColor(originalmage, cv2.COLOR_BGR2GRAY)
    ReSized2 = cv2.resize(grayScaleImage, (x, y))
    #plt.imshow(ReSized2, cmap='gray')

    #applying median blur to smoothen an image
    smoothGrayScale = cv2.medianBlur(grayScaleImage, 5)
    ReSized3 = cv2.resize(smoothGrayScale, (x, y))
    #plt.imshow(ReSized3, cmap='gray')

    #retrieving the edges for cartoon effect
    #by using thresholding technique
    getEdge = cv2.adaptiveThreshold(smoothGrayScale, 255, 
    cv2.ADAPTIVE_THRESH_MEAN_C, 
    cv2.THRESH_BINARY, 9, 9)
    ReSized4 = cv2.resize(getEdge, (x, y))
    #plt.imshow(ReSized4, cmap='gray')

    #applying bilateral filter to remove noise 
    #and keep edge sharp as required
    colorImage = cv2.bilateralFilter(originalmage, 9, 300, 300)
    ReSized5 = cv2.resize(colorImage, (x, y))
    #plt.imshow(ReSized5, cmap='gray')

    #masking edged image with our "BEAUTIFY" image
    cartoonImage = cv2.bitwise_and(colorImage, colorImage, mask=getEdge)
    ReSized6 = cv2.resize(cartoonImage, (x, y))
    #plt.imshow(ReSized6, cmap='gray')

    # Plotting the whole transition
    # images=[ReSized1, ReSized2, ReSized3, ReSized4, ReSized5, ReSized6]
    # fig, axes = plt.subplots(3,2, figsize=(8,8), subplot_kw={'xticks':[], 'yticks':[]}, gridspec_kw=dict(hspace=0.1, wspace=0.1))
    # for i, ax in enumerate(axes.flat):
    #     ax.imshow(images[i], cmap='gray')
    
    # plt.show()

    return cartoonImage

In [16]:
def save(Image, ImagePath):

    #saving an image using imwrite()
    n = ImagePath.split("\\")
    n = n[-1].rsplit(".",1)
    newName= n[0] +"_cartoonified"
    path1 = os.path.dirname(ImagePath)
    extension=os.path.splitext(ImagePath)[1]
    path = os.path.join(path1, newName + extension)
    cv2.imwrite(path, cv2.cvtColor(Image, cv2.COLOR_RGB2BGR))

    #print(path)

    # I = "Image saved by name " + newName +" at "+ path
    # tk.messagebox.showinfo(title=None, message=I)
   

## Hilfsklassen


In [17]:
class myFaceImageDataset(torch.utils.data.Dataset):
    def __init__(self, dataset_dict, transpose=False, transform = None):
        #das Übergeben Dict mit den Daten hat folgende Form:
        #{ key: index , value: list[ sample, label]}
        self.dataset_dict = dataset_dict
        self.transpose = transpose

        self.transform = transform

    def __len__(self):
        return len(self.dataset_dict)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        sample, target = self.dataset_dict[idx]

        if self.transpose:
            #Transformiere das Sample, so dass es 
            sample = sample.transpose(-1,0)

        if self.transform != None:
            sample = self.transform(sample)

        sample = torch.tensor(sample, dtype=torch.float32)

        return (sample, target)

______________________________
## Pipeline für den Datensatz

Der genutzte Datensatz ist auf Kaggle zu finden. \
(https://www.kaggle.com/datasets/sudarshanvaidya/random-images-for-face-emotion-recognition?resource=download).

In [18]:
def load_data(path_folder, add_RGB=False):
    #Loading the data into a the Notebook
    data = {}
    count = 0

    for t in targets:    
        p = path_folder + "/" + t
        #print(t)
        for filename in os.listdir(p):

            f = os.path.join(p,filename)
            if os.path.isfile(f):

                # the sample is save on the 0. position and the target on the 1. position
                x = torch.tensor(plt.imread(f), dtype=float)

                #print(count)
                #print(filename)
                if add_RGB: 
                    x = add_RGBLayers(x)

                data[count] = (x, tar_name2vec[t])

                count += 1

    return data

In [19]:
def image_transformation_pipeline(path, transformation="CartCV2"):
    
    
    #Files and directories in a specified path

    #Hier wird über jede Datei iteriert, transformiert und in einen neuen Ordner gespeichert
    for filename in os.listdir(path):

        f = os.path.join(path,filename)
        if os.path.isfile(f):

            #nimmt den vorigen Path und passt in dann an
            fn = f.split("\\")

            #hier wird Entschieden welche Vorverarbeitung gewählt wird
            if transformation == "CartCV2":

                #Entnimmt aus dem alten Pfad f
                test = cartoonify(f)

                #print(f)

                #Die Ordner existieren bereits
                fn[0] = fn[0] + "_Cartoonified_OpenCV"
                fn = "\\".join(fn)

                #print(fn)
                #Steckt es in neuen Pfad fn
                save(test, fn)

            # if transformation == "DualGAN":

                # test = dualStyleGAN_transform(f)

                # fn[0] = fn[0] + "_Cartoonified_OpenCV"
                # fn = "\\".join(fn)
                
                #print(fn)
                #Steckt es in neuen Pfad fn
                # save(test, fn)

                # TODO
    

In [20]:
# for i in targets:
#     p = "Trainingsbilder\\" + i
#     image_transformation_pipeline(p)

so apparently all images in the dataset have the same size, which makes things easier

# Setting up the Model

Dokumentation dazu: https://pytorch.org/vision/0.11/models.html

In [21]:
# Definition of the net structure
class Net(torch.nn.Module):
    def __init__(self, n_classes):
        super(Net, self).__init__()
        self.n_classes = n_classes

        # Resnet Featureextractor LAden, ich werfe hier den großen LinearLayer am Ende von dem Resnet weg
        self.fe = torch.nn.Sequential(*(list(torchvision.models.resnet18(pretrained=True).children())[:-1]))
        # Die Featur Map die aus dem Resnet raus kommt hat eine Größe von batch_size x 512 x 1 x 1
        # -> Das bringen wir nacher in der forward Mehtode noch auf die richtige Dimension

        # dein Lienar Layer muss aber 512 Eingänge haben
        self.fe2 = torch.nn.Sequential(torch.nn.Linear(512, 16),
                                        torch.nn.ReLU(),
                                        torch.nn.Dropout(),
                                        torch.nn.Linear(16, self.n_classes))

        # Dropout sorgt für Regularisierung, gegen Overfitting

    def forward(self, x):
        # den squeeze braucht man, weil die Feature Map eine Größe von batch_size x 512 x 1 x 1 hat,
        # die wird damit reduziert auf batch_size x 512
        y_pred = self.fe2(self.fe(x).squeeze())

        return y_pred


# Modelltraining

Interessante Dokumentationen:\
[BCEWithLogitsLoss — PyTorch 1.11.0 documentation](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)\
[torch.nn.functional.one_hot — PyTorch 1.11.0 documentation](https://pytorch.org/docs/stable/generated/torch.nn.functional.one_hot.html)\
https://pytorch.org/docs/stable/data.html \
https://pytorch.org/tutorials/beginner/data_loading_tutorial.html \
https://pytorch.org/tutorials/beginner/basics/data_tutorial.html \
[Data Augmentation](https://pytorch.org/tutorials/beginner/basics/transforms_tutorial.html)\
[Optimizing Model Parameters](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html)\
[Modelle einfrieren](https://pytorch.org/docs/master/notes/autograd.html)

Falls der DataLoader rummeckert müsste ich mir eine "costum collate fnc" erstellen (ins Dataset)


In [22]:
def train(net, train_l, test_l, optim):
    crit =  torch.nn.BCEWithLogitsLoss()


    bce_train = []
    bce_test = []
        
    
    
    for epoch in tqdm(range(0, epochen )):
        net.train()
        summ = []
        c = 0
        for b_idx, (dat, tar) in enumerate(train_l):
            # --- Training ----#
            optim.zero_grad()
            y_pred = net(dat)
            train_loss = crit(y_pred, tar)
            train_loss.backward()
            optim.step()
            

            # ----- Evaluation metrics train ---- #
            summ.append(train_loss.item())
            c += 1
                
        bce_train.append(sum(summ)/c)
        

        
        # ----- Test Prediction & Evaluation -------------#
        bce_test.append(calc_loss(net, test_l)) 
        
    net.eval()

    return net, bce_train, bce_test    

____________________________________________
### Vorverarbeitungsschritte

In [23]:
# for i in targets:
#     p = "Trainingsbilder\\" + i
#     image_transformation_pipeline(p)

In [24]:
# augmentation_pipeline_forFolder("J:\\Seminararbeit\\Trainingsbilder",
#                                 "J:\\Seminararbeit\\Trainingsbilder+Augmentations", True)

# brauch ich nicht mehr, da ich die Augmentation ins Dataset verlagere

__________________________

In [25]:
learning_rate = 0.001
batch_size = 16
epochen = 2

In [26]:
aug_transformations = T.Compose([
    T.ColorJitter(brightness=(0.8,1.2)),
    T.RandomVerticalFlip(),
    T.RandomRotation(degrees=45),
    T.RandomErasing(),
    T.RandomCrop(size=(180,180), fill=0, padding_mode="constant"),
    T.Resize(size=(224,224))
])

In [27]:
data = myFaceImageDataset(load_data(path_folder, add_RGB=True), transpose=True)

data_train, data_val = my_train_validation_split(data, 0.9, shuffle=True, trans=aug_transformations)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [28]:
my_model = Net(8)

#friert das pretrained ResNet ein
for param in my_model.fe.children():
    param.requires_grad_ = False

In [29]:
optimizer = torch.optim.Adam(my_model.parameters(), lr=learning_rate)

# Note: Daten noch in Train und Test unterteilen und in ein TorchDataset transformieren
train_loader = torch.utils.data.DataLoader(
    data_train,
    batch_size = batch_size,
    shuffle = True
)

val_loader = torch.utils.data.DataLoader(
    data_val,
    batch_size = len(data_val),
    shuffle = False
)

In [ ]:
my_model, bce_train, bce_test = train(my_model, train_loader, val_loader, optimizer)

  0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [ ]:
fig, ax = plt.subplots(1,2, figsize=(18,6))

ax[0].plot(bce_train,        color="navy", label="Train Error")
ax[0].set_xlabel("Anzahl an Updates")
ax[0].set_ylabel("Trainingsfehler")

ax[1].plot(bce_test,  "r--", color="navy", label="Test Error" )
ax[1].set_xlabel("Anzahl an Epochen")
ax[1].set_ylabel("Testfehler")

plt.show()

## Testgebiet

In [ ]:
# torch.save(my_model.state_dict(), "Baselinemodell")

In [ ]:
# torch.save(data_train, "Trainingsbilder/Train")
# torch.save(data_val, "Trainingsbilder/Validation")

In [ ]:
loaded_model = Net(8)
loaded_model.load_state_dict(torch.load("Modell1/Baselinemodell"))


In [ ]:
d = data[0][0]
print(d.shape)
d = torch.unsqueeze(d, 0)
print(d.shape)

# Net().forward(d)

In [ ]:
test = cv2.imread("Trainingsbilder/happiness/7647334524_7fe96ff1f0_n_face.png")
print(test.shape)
plt.imshow(test, cmap="gray")
# Alle Bilder sind nur Graubilder
# habe keine 3. Dimesion -> könnten also falsch interpretiert werden.
# D.h. als zusätzlichen Vorverabeitungsschritt sollte ggf. RGB hinzugefügt werden

In [ ]:
#test= add_RGBLayers(test)
plt.imshow(test)

In [ ]:
lowpass = ndimage.gaussian_filter(test, 3)
gauss_highpass = (test - lowpass + np.uint8(np.ones_like(test)* 1))

print(gauss_highpass.shape)

plt.imshow(add_RGBLayers(gauss_highpass))
print(np.mean(gauss_highpass))
print(np.mean(test))

In [ ]:
def black_white_transform(v, mean=100):
    if v < mean:
        return 0
    else:
        return 1

In [ ]:
tfunc = np.vectorize(black_white_transform)
test = tfunc(test)
plt.imshow(test)

In [ ]:
cim = cartoonify("Trainingsbilder/happiness/7647334524_7fe96ff1f0_n_face.png")

cim.shape

In [ ]:
plt.imshow(cim)
cim = torch.tensor(cim, dtype=torch.float32)

In [ ]:
(cim.transpose(-1,0)).shape

In [ ]:
y = loaded_model(cim.transpose(-1,0).unsqueeze(dim=0))

In [ ]:
print(torch.softmax(y, dim=0, dtype=torch.float32))
print(torch.max(torch.softmax(y, dim=0, dtype=torch.float32)))

In [ ]:
cim2 = plt.imread("Trainingsbilder_Cartoonified_OpenCV/contempt/images - 2020-11-06T181758.699_face_cartoonified.png")
plt.imshow(cim2)

In [ ]:
cim2 = torch.tensor(cim2, dtype = torch.float32)
(cim2.transpose(-1,0)).shape

In [ ]:
y = loaded_model(cim2.transpose(-1,0).unsqueeze(dim=0))

In [ ]:
print(torch.softmax(y, dim=0, dtype=torch.float32))
print(torch.max(torch.softmax(y, dim=0, dtype=torch.float32)))

In [ ]:
drawn = torch.tensor(plt.imread("Testbilder/yesyesyes w.png"), dtype=torch.float32)
#plt.imshow(drawn)
drawn = drawn[:,:,:3]
print(drawn.shape)
plt.imshow(drawn)

In [ ]:
drawn_predict = loaded_model(drawn.transpose(-1,0).unsqueeze(dim=0))
print(torch.softmax(drawn_predict, dim=0, dtype=torch.float32))
print(torch.max(torch.softmax(drawn_predict, dim=0, dtype=torch.float32)))

In [ ]:
drawn = torch.tensor(plt.imread("Testbilder/2TManager.png"), dtype=torch.float32)
#plt.imshow(drawn)
drawn = drawn[:,:,:3]
print(drawn.shape)
plt.imshow(drawn)

In [ ]:
drawn_predict = loaded_model(drawn.transpose(-1,0).unsqueeze(dim=0))
print(torch.softmax(drawn_predict, dim=0, dtype=torch.float32))
print(torch.max(torch.softmax(drawn_predict, dim=0, dtype=torch.float32)))

In [ ]:
#save(cim, "Trainingsbilder_Cartoonified_OpenCV/happiness/7647334524_7fe96ff1f0_n_face.png")


In [ ]:
a = "Trainingsbilder_Cartoonified_OpenCV/happiness/7647334524_7fe96ff1f0_n_face.png".split("/")
a = a[-1].split(".")
a = a[0] +"_cartoonified."+ a[1]
a

In [ ]:
originalmage = cv2.imread("Trainingsbilder\\happiness\\7647334524_7fe96ff1f0_n_face.png")
plt.imshow(originalmage)
print(originalmage.shape)
x, y, c = originalmage.shape

In [ ]:
path_of_the_directory= "Trainingsbilder\happiness"
print("Files and directories in a specified path:")
for filename in os.listdir(path_of_the_directory):
    f = os.path.join(path_of_the_directory,filename)
    if os.path.isfile(f):
        print(f)
        fn = f.split("\\")
        fn[0] = fn[0] + "_Cartoonified_OpenCV"
        fn = "/".join(fn)
        print(fn)

In [ ]:
# teste alle Augmetations an Lena aus
ttt = torch.tensor(plt.imread("images - 2020-11-06T002957.309_face.png"))
ttt = add_RGBLayers(ttt)
plt.imshow(ttt)

a = augmentation_pipeline_forOneImage(ttt)

print(len(a))
for i in a:
    plt.imshow(i, cmap="gray")
    plt.show()